## Object Detection

In [ ]:
import json
import torch

from os import listdir, makedirs, path

from PIL import Image as PImage

from dominant_colors import get_dominant_colors, resize_PIL

IMAGES_IN_PATH = "../../imgs/arquigrafia"

OUT_PATH = "./metadata/json/objects"
makedirs(OUT_PATH, exist_ok=True)

In [ ]:
OBJS = [
  {
    "minaret": 0.25,
    "tower": 0.6,
    "railing": 0.4,
    "stair railing": 0.41,
    "guard railing": 0.4,
    "table": 0.45,
    "desk": 0.25,
    "chair": 0.24,
    "inclined walkway": 0.32,
    "sculpture": 0.4,
    "painting": 0.4,
    "vertical pillar": 0.35,
    "stairs": 0.4,
    "stoop steps": 0.35,
    "stoop stairs": 0.35,
  },
  {
    "window": 0.2,
    "room door": 0.25,
    "building door": 0.22,
    "masonry": 0.2,

    "concrete wall": 0.2,
    "exposed concrete": 0.2,
    "concrete structure": 0.2,
    "poured concrete": 0.2,
  
    "glass window": 0.2,
    "glass door": 0.2,
    "mirror": 0.2,
  },
  {
    "wood fence": 0.3,
    "wood railing": 0.35,
    "wood pilar": 0.3,
    "wood door": 0.21,
    "wood board": 0.21,

    "metal fence": 0.4,
    "metal railing": 0.22,
    "wrought": 0.2,
  },
  {
    "tree": 0.2,
    "grass": 0.2,
    "shrub": 0.2,
    "bush": 0.2,
    "flower": 0.2,
    "vegetation": 0.2,
    "greenery": 0.2,
  }
]

LABEL2LABEL = {
  "minaret": "tower",
  "stair railing": "railing",
  "guard railing": "railing",
  "stoop steps": "stairs",
  "stoop stairs": "stairs",
  "desk": "table",
  "room door": "building door",

  "exposed concrete": "concrete wall",
  "concrete structure": "concrete wall",
  "poured concrete": "concrete wall",

  "glass window": "mirror",
  "glass door": "mirror",

  "wood railing": "wood fence",
  "wood pilar":"wood fence",
  "wood door": "wood fence",
  "wood board": "wood fence",

  "metal fence": "wrought",
  "metal railing": "wrought",

  "tree": "greenery",
  "grass": "greenery",
  "shrub": "greenery",
  "bush": "greenery",
  "flower": "greenery",
  "vegetation": "greenery",
}

OBJS_LABELS_IN = [sorted(o.keys()) for o in OBJS]
OBJS_LABELS_OUT = [[LABEL2LABEL.get(l, l) for l in oli] for oli in OBJS_LABELS_IN]
OBJS_THOLDS = [[OBJS[i][k] for k in oli] for i,oli in enumerate(OBJS_LABELS_IN)]

### Helper Functions

In [ ]:
from transformers import Owlv2Processor, Owlv2ForObjectDetection

OBJ_TARGET_SIZE = torch.Tensor([500, 500])
OBJ_MODEL = "google/owlv2-base-patch16-ensemble"

obj_model = Owlv2ForObjectDetection.from_pretrained(OBJ_MODEL).to("cuda")
obj_processor = Owlv2Processor.from_pretrained(OBJ_MODEL)

In [ ]:
def box_px_to_pct(box, img_w, img_h, model_dims):
  scale_factor = torch.tensor([max(img_w, img_h) / img_w , max(img_w, img_h) / img_h])
  return [round(x, 4) for x in (box.cpu().reshape(2, -1) / model_dims * scale_factor).reshape(-1).tolist()]

In [ ]:
def run_object_detection(img, obj_labels_in, obj_labels_out, obj_tholds):
  input = obj_processor(text=obj_labels_in, images=img, return_tensors="pt").to("cuda")
  with torch.no_grad():
    obj_out = obj_model(**input)

  res = obj_processor.post_process_object_detection(outputs=obj_out, target_sizes=[OBJ_TARGET_SIZE])
  slbs = zip(res[0]["scores"], res[0]["labels"], res[0]["boxes"])
  iw, ih = img.size

  # filter if box "too large" or "too small"
  def good_thold_and_size(s, l, b):
    box_pct = box_px_to_pct(b, iw, ih, OBJ_TARGET_SIZE)
    box_width = box_pct[2] - box_pct[0]
    box_height = box_pct[3] - box_pct[1]
    good_min = box_width > 0.05 and box_height > 0.05
    good_max = box_width < 0.8 or box_height < 0.8
    return good_min and good_max and s > obj_tholds[l.item()]

  detected_objs = [{"score": s, "label": obj_labels_out[l.item()], "box": box_px_to_pct(b, iw, ih, OBJ_TARGET_SIZE)} for s,l,b in slbs if good_thold_and_size(s, l, b)]

  # only keep the box with highest score per object
  detected_objs_boxes = {}
  high_score = {}

  for o in detected_objs:
    ol = o["label"]
    if (ol not in detected_objs_boxes) or (o["score"] > high_score[ol]):
      detected_objs_boxes[ol] = o["box"]
      high_score[ol] = o["score"]

  return detected_objs_boxes

### Run Object Detection

In [ ]:
%%time

input_files = sorted([f for f in listdir(IMAGES_IN_PATH) if f.endswith("jpg")])

for io_file in input_files[:4096]:
  input_file_path = path.join(IMAGES_IN_PATH, io_file)
  output_file_path = path.join(OUT_PATH, io_file.replace(".jpg", ".json"))

  if path.isfile(output_file_path):
    continue

  if int(io_file.replace(".jpg", "")) % 50 == 0:
    print(IMAGES_IN_PATH, io_file)

  image = PImage.open(input_file_path).convert("RGB")

  rgb_by_count, rgb_by_hls = get_dominant_colors(resize_PIL(image))

  image_data = {}

  image_data["boxes"] = {}
  for i in range(0, len(OBJS_LABELS_IN)):
    obj_boxes = run_object_detection(image, OBJS_LABELS_IN[i], OBJS_LABELS_OUT[i], OBJS_THOLDS[i])
    image_data["boxes"] = image_data["boxes"] | obj_boxes

  image_data["dominant_color"] = {
    "by_count": [int(v) for v in rgb_by_count[0]],
    "by_hue": [int(v) for v in rgb_by_hls[0]],
    "palette": [[int(v) for v in c] for c in rgb_by_hls[:4]],
  }

  with open(output_file_path, "w", encoding="utf-8") as of:
    json.dump(image_data, of, sort_keys=True, separators=(',',':'), ensure_ascii=False)

### Post-Process: Create output json file

In [ ]:
from export_utils import export_objs_caps

OBJECTS_PATH = "./metadata/json/objects"
CAPTIONS_PATH = "./metadata/json/captions"
OBJECTS_DB_FILE_PATH = "./metadata/json/objects.json"

export_objs_caps(OBJECTS_PATH, CAPTIONS_PATH, OBJECTS_DB_FILE_PATH)

### Post-Process: Create separate json files

In [ ]:
from export_utils import export_by_keys

OBJECTS_PATH = "./metadata/json/objects"

keys = ["binaries", "boxes"]
export_by_keys(OBJECTS_PATH, keys)

### Count objects

In [ ]:
OBJECTS_DB_FILE_PATH = "./metadata/json/objects.json"
IMAGES_PATH = "../../imgs/arquigrafia"

LABEL2LABEL = {
  "tower": "chimney/tower",
  "railing": "railing/banister",
  "inclined walkway": "ramp",
  "stairs": "stairs",
  "table": "table",
  "building door": "door",
  "concrete wall": "concrete",
  "masonry": "masonry",
  "mirror": "glass",
  "wood fence": "wood",
  "wrought": "metal",
  "greenery": "vegetation",
  "chair": "chair",
  "painting": "painting",
  "sculpture": "sculpture",
  "vertical pillar": "pillar",
  "window": "window",
}

with open(OBJECTS_DB_FILE_PATH, "r") as f:
  json_data = json.load(f)
  img_data = json_data["images"]
  obj_data = json_data["objects"]

object_count = sorted([(LABEL2LABEL[k], len(a)) for k, a in obj_data.items()], key=lambda x: x[1], reverse=True)

for o,c in object_count:
  print(o, ":", c)


### TEST: boxes from JSON

In [ ]:
import json

from os import path
from PIL import Image as PImage, ImageDraw as PImageDraw, ImageFont as PImageFont

MFONT = PImageFont.load_default(20)

OBJECTS_DB_FILE_PATH = "./metadata/json/objects.json"
IMAGES_PATH = "../../imgs/arquigrafia"

with open(OBJECTS_DB_FILE_PATH, "r") as f:
  json_data = json.load(f)
  img_data = json_data["images"]
  obj_data = json_data["objects"]

for id, d in list(img_data.items())[:3]:
  img_path = path.join(IMAGES_PATH, f"{id}.jpg")
  img = PImage.open(img_path).convert("RGBA")
  iw,ih = img.size
  draw = PImageDraw.Draw(img)
  for label, (x0,y0,x1,y1) in d["boxes"].items():
    draw.rectangle(((x0*iw, y0*ih), (x1*iw, y1*ih)), outline=(255, 0, 0), width=2)
  print(list(d["boxes"].keys()), "\n", d["caption"])
  display(img)

### TEST: Dominant Color

In [ ]:
import numpy as np

from os import listdir, path
from PIL import Image as PImage

from dominant_colors import get_dominant_colors, resize_PIL

IMAGES_IN_PATH = "../../imgs/arquigrafia"
INPUT_FILES = sorted([f for f in listdir(IMAGES_IN_PATH) if f.endswith("jpg")])

io_file = INPUT_FILES[0]
input_file_path = path.join(IMAGES_IN_PATH, io_file)

In [ ]:
image = PImage.open(input_file_path).convert("RGB")
image_s = resize_PIL(image)
rgb_by_count, rgb_by_hls = get_dominant_colors(image_s)

In [ ]:
iw, ih = [(d // 2) * 2 for d in image_s.size]
image_shape = (ih, iw, 3)
ppc = int(ih * iw / len(rgb_by_count))

count_np_image = np.array([ppc * [c] for c in rgb_by_count]).reshape(image_shape)
hls_np_image = np.array([ppc * [c] for c in rgb_by_hls]).reshape(image_shape)

display(image_s)
display(PImage.fromarray(count_np_image))
display(PImage.fromarray(hls_np_image))

### Adjust Thresholds

In [ ]:
import json
import shutil

from os import path, listdir, makedirs

OBJECTS_DB_FILE_PATH = "./metadata/json/objects.json"
IMAGES_PATH = "../../imgs"
IMG_IN_DIR = "arquigrafia"

with open(OBJECTS_DB_FILE_PATH, "r") as f:
  json_data = json.load(f)
  img_data = json_data["images"]
  obj_data = json_data["objects"]

print(obj_data.keys(), len(obj_data.keys()))

In [ ]:
tocopy = [
  'inclined walkway',
]

for o in tocopy:
  print(o, len(obj_data[o]), obj_data[o], "\n")
  img_out_dir = f"test-{o.replace(' ', '-')}"
  img_out_dir_path = path.join(IMAGES_PATH, img_out_dir)
  makedirs(img_out_dir_path, exist_ok=True)
  for i in obj_data[o]:
    img_in_path = path.join(IMAGES_PATH, IMG_IN_DIR, f"{i}.jpg")
    shutil.copy2(img_in_path, img_out_dir_path)

In [ ]:
from PIL import Image as PImage, ImageDraw as PImageDraw, ImageFont as PImageFont

MFONT = PImageFont.load_default(20)

TEST_PATH = "../../imgs/test-inclined-walkway"

OBJS = {
  "inclined walkway": 0.41,
}

LABEL2LABEL = {}

OBJS_LABELS_IN = sorted(OBJS.keys())
OBJS_LABELS_OUT = [LABEL2LABEL.get(l, l) for l in OBJS_LABELS_IN]
OBJS_THOLDS = [OBJS[k] for k in OBJS_LABELS_IN]

In [ ]:
def run_object_detection(img, obj_labels_in, obj_labels_out, obj_tholds):
  input = obj_processor(text=obj_labels_in, images=img, return_tensors="pt").to("cuda")
  with torch.no_grad():
    obj_out = obj_model(**input)

  res = obj_processor.post_process_object_detection(outputs=obj_out, target_sizes=[OBJ_TARGET_SIZE])
  slbs = zip(res[0]["scores"], res[0]["labels"], res[0]["boxes"])
  iw, ih = img.size

  # filter if box "too large" or "too small"
  def good_thold_and_size(s, l, b):
    box_pct = box_px_to_pct(b, iw, ih, OBJ_TARGET_SIZE)
    box_width = box_pct[2] - box_pct[0]
    box_height = box_pct[3] - box_pct[1]
    good_min = box_width > 0.05 and box_height > 0.05
    good_max = box_width < 0.8 or box_height < 0.8
    return good_min and good_max and s > obj_tholds[l.item()]

  detected_objs = [{"score": s.item(), "label": obj_labels_out[l.item()], "box": box_px_to_pct(b, iw, ih, OBJ_TARGET_SIZE)} for s,l,b in slbs if good_thold_and_size(s, l, b)]

  # only keep the box with highest score per object
  detected_objs_boxes = {}
  high_score = {}

  for o in detected_objs:
    ol = o["label"]
    if (ol not in detected_objs_boxes) or (o["score"] > high_score[ol]):
      detected_objs_boxes[ol] = o["box"]
      high_score[ol] = o["score"]

  return detected_objs #detected_objs_boxes

In [ ]:
ppath = TEST_PATH
input_files = sorted([f for f in listdir(ppath) if f.endswith("jpg")])

for io_file in input_files:
  input_file_path = path.join(ppath, io_file)

  image = PImage.open(input_file_path).convert("RGB")
  iw,ih = image.size
  print(image.size)

  objs = run_object_detection(image, OBJS_LABELS_IN, OBJS_LABELS_OUT, OBJS_THOLDS)
  print([f'{o["label"]}: {o["score"]}' for o in objs])

  draw = PImageDraw.Draw(image)
  for o in objs:
    (x0,y0,x1,y1) = o["box"]
    score, label = o["score"], o["label"]
    draw.rectangle(((x0*iw, y0*ih), (x1*iw, y1*ih)), outline=(255, 0, 0), width=2)
    draw.text((x0*iw, y0*ih + 20), f"{round(score, 3)}", (255, 255, 255), font=MFONT)
    draw.text((x0*iw, y0*ih - 0), f"{label}", (255, 0, 0), font=MFONT)
  display(image)